## 1. Notebook Settings

# Setup

In [1]:
import os
import sys
from pathlib import Path

import plotly.graph_objects as go
import plotly.io as pio
from dotenv import load_dotenv

# Add the src directory to the Python path
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from src.maritime_module.core.graph import (FineGraph, GraphConfigManager,
                                            H3Graph)
from src.maritime_module.core.pathfinding_lite import Route
from src.maritime_module.core.s57_data import (ENCDataFactory,
                                                S57AdvancedConfig)
from src.maritime_module.utils.geometry_utils import Buffer, Slicer
from src.maritime_module.utils.plot_utils import PlotlyChart
from src.maritime_module.utils.port_utils import Boundaries, PortData

# Load environment variables from .env file at the project root
load_dotenv(project_root / ".env")
pio.renderers.default = "notebook_connected"

# Define paths for data and output

output_dir = Path.cwd() / 'output'
output_dir.mkdir(exist_ok=True)

# Define database parameters from environment variables
db_params = {
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT')
}

print(f"Output directory: {output_dir}")
print(f"PostGIS DB_name: {db_params['dbname']}")

Output directory: /home/vikont_tux/python_projects_wsl2/1_MaritimeModule_V1/docs/notebooks/output
PostGIS DB_name: ENC_db


## Setup PostGIS DataFactory

In [2]:
pg_factory = ENCDataFactory(source=db_params, schema="us_enc_all")
buffer_sliced = True
graph_mode = "h3" # "fine" or "h3"
save_gpkg = True
save_pg = True
save_pg_opt = True
calc_route = False

2025-10-15 17:43:36,712 - src.maritime_module.core.s57_data - INFO - Source is a dictionary, initializing PostGISManager.
2025-10-15 17:43:36,842 - src.maritime_module.core.s57_data - INFO - Successfully connected to database 'ENC_db'


## Initialize Plotly

In [3]:
ply = PlotlyChart()
ply_fig = ply.create_base_map(mapbox_token="pk.eyJ1Ijoidmlrb250IiwiYSI6ImNtNHljdzQwbjBybWIybHNkZzhleXhqYXUifQ.Jz0cR4J6PdAnLutVBqj2WA")
ply.plotly_base_config(ply_fig)

{'displayModeBar': 'hover',
 'responsive': True,
 'scrollZoom': True,
 'displaylogo': False,
 'modeBarButtonsToRemove': ['zoomIn',
  'zoomOut',
  'pan',
  'select',
  'lassoSelect'],
 'modeBarButtonsToAdd': ['autoScale', 'hoverCompareCartesian']}

# FineGraph adn H3Graph workflow

## Import Route

In [4]:
base_route = pg_factory.load_route(route_name= "base_Route",
                      schema_name= "routes",
                      table_name= "base_route")
# This performs a deep copy, ensuring that changes to the copy do not affect the original.
ply_route = go.Figure(ply_fig)

ply.add_route_trace(figure=ply_route,
                    line=base_route,
                    name="Base Route")
ply_route.show()

#### Create Buffer

In [5]:
route_buffer = Buffer.create_buffer(base_route, 24.0)
print(type(route_buffer))

2025-10-15 17:43:37,057 - src.maritime_module.utils.geometry_utils - INFO - Creating buffer of 24.0 nm (0.400000 degrees) for LineString.
<class 'shapely.geometry.polygon.Polygon'>


In [6]:
ply.add_polygon_trace(fig=ply_route,
                      polygon=route_buffer,
                      name="Base Route Buffer")
ply_route.show()

#### Slice Buffer (to reduce area of interest)

In [7]:
sliced_buffer = Slicer.slice_by_bbox(route_buffer, south=37.0)
ply.add_polygon_trace(fig=ply_route,
                      polygon=sliced_buffer,
                      name="Sliced Buffer")
ply_route.show()

2025-10-15 17:43:37,205 - src.maritime_module.utils.geometry_utils - INFO - Slicing Polygon with bounding box: N=38.2167, E=-117.8370, S=37.0000, W=-122.9220


## Graph Creation Prerequasits

#### Define geographic points of Departure and Arrival.

##### Point can be taken from World Port Index or Custom User Defined Points

In [8]:
port = PortData()
port.create_custom_port(port_name="Pilot",
                        lon=-122.27,
                        lat=37.0,
                        if_exists='update')
dep_point = port.get_port_by_name("Pilot")
arr_point = port.get_port_by_name("San Francisco")
port1_df = port.get_port_details_df(dep_point)
port2_df = port.get_port_details_df(arr_point)
print(port.format_port_string(dep_point))
print(port.format_port_string(arr_point))
ply.add_single_port_trace(ply_route, dep_point, name=dep_point['PORT_NAME'], color='blue')
ply.add_single_port_trace(ply_route, arr_point, name=arr_point['PORT_NAME'], color='red')
ply_route.show()

2025-10-15 17:43:37,717 - src.maritime_module.utils.port_utils - INFO - Loaded 7 custom ports. Merging with standard ports.
2025-10-15 17:43:37,738 - src.maritime_module.utils.port_utils - INFO - Port 'Pilot' exists. Updating.
2025-10-15 17:43:37,754 - src.maritime_module.utils.port_utils - INFO - Updated attributes for custom port 'Pilot'.
2025-10-15 17:43:37,763 - src.maritime_module.utils.port_utils - INFO - Loaded 7 custom ports. Merging with standard ports.
PILOT, CUSTOM (LAT: 37° 0.0' N  LONG: 122° 16.19999999999976' W)
SAN FRANCISCO, US (LAT: 37° 49.0' N  LONG: 122° 25.0' W)


#### Config Graph parameters and create ENC list

In [9]:
if buffer_sliced:
    route_buffer = sliced_buffer
else:
    route_buffer = route_buffer

config_path = project_root / "src/maritime_module/data/graph_config.yml"
config_manager = GraphConfigManager(config_path)

enc_list = pg_factory.get_encs_by_boundary(route_buffer)

2025-10-15 17:43:37,867 - src.maritime_module.core.s57_data - INFO - Factory: Filtering ENCs by boundary...
2025-10-15 17:43:37,867 - src.maritime_module.core.s57_data - INFO - Factory: Getting ENC summary...
2025-10-15 17:43:37,893 - src.maritime_module.core.s57_data - INFO - Factory: Getting bounding boxes for 6369 ENCs...


## Fine Graph

#### Fine Grid Creation

In [10]:
# Get the layer settings from the config
layers_config = config_manager.get_value("layers")

# Extract the specific lists for navigable and obstacle layers
navigable_layers_config = layers_config.get('navigable', [])
obstacle_layers_config = layers_config.get('obstacles', [])

if graph_mode == "fine":
    fg = FineGraph(data_factory=pg_factory,
                   route_schema_name="routes",
                   graph_schema_name="graph")

    fg_grid = fg.create_fine_grid(route_buffer=route_buffer,
                                  enc_names=enc_list,
                                  navigable_layers=navigable_layers_config,
                                  obstacle_layers=obstacle_layers_config
                                  )

#### Plot Fine Grid

In [11]:
if graph_mode == "fine":
    ply_fine_grid = go.Figure(ply_fig)
    ply.add_grid_trace(ply_fine_grid, grid_geojson=fg_grid["main_grid"], color="blue")
    ply.add_grid_trace(ply_fine_grid, grid_geojson=fg_grid["combined_grid"], color="red")
    ply_fine_grid.show()

#### Fine Graph Creation

In [12]:
# Time: 1m 12s (Buffer-24NM, Spacing-0.3Nm) nodes: 158,803 edges: 627,475
# Time: 2m 43s (Buffer-24NM, Spacing-0.2Nm)
# Time: 5m 59s (Buffer-24NM, Spacing-0.1Nm)
if graph_mode == "fine":
    fg_space = 0.2
    G_fine = fg.create_base_graph(grid_data=fg_grid["combined_grid"],
                         spacing_nm=fg_space,
                         max_points=4500000,
                         keep_largest_component=True)

## H3 Graph

#### H3 Grid and Graph Creation

In [13]:
# Time: 1m 58s (Buffer-24NM, Spacing-6-11) hexagons: 933,188 nodes: 933,188 edges: 2,765,899
# Time: 9m 1s (Buffer-24NM, Spacing-6-11) hexagons: 3,841,847 nodes: 3,841,847 edges: 11,429,039
# Time: 9m 1s (Buffer-24NM, Spacing-6-11) hexagons: 3,841,847 nodes: 3,841,847 edges: 11,533,570 (Bridge)
# Time: 13m 58s (Buffer-24NM, Spacing-6-11) hexagons: 3,841,847 nodes: 3,838,809 edges: 11,528,770 total_components: 462(Bridge_component)
# Time: 13m 20s (Buffer-24NM, Spacing-6-11) hexagons: 3,841,847 nodes: 3,838,809 edges: 11,528,770 total_components: 462(Bridge_component)

if graph_mode == "h3":
    h3 = H3Graph(data_factory=pg_factory,
                route_schema_name="routes",
                graph_schema_name="graph")

    h3_settings = config_manager.get_value("h3_settings")
    connectivity_config = h3_settings.get('connectivity', {})

    G_h3, h3_grid = h3.create_h3_graph(route_buffer=route_buffer,
                                       enc_names=enc_list,
                                       navigable_layers=navigable_layers_config,
                                       obstacle_layers=obstacle_layers_config,
                                       connectivity_config=connectivity_config,
                                       keep_largest_component=True)

2025-10-15 17:43:38,588 - src.maritime_module.core.graph - INFO - Starting H3 grid creation from configuration.
2025-10-15 17:43:38,589 - src.maritime_module.core.s57_data - INFO - Factory: Getting layer 'lndare'...
2025-10-15 17:43:38,911 - src.maritime_module.core.s57_data - INFO - Factory: Getting layer 'lndare'...
2025-10-15 17:43:38,976 - src.maritime_module.core.s57_data - INFO - Factory: Getting layer 'lndare'...
2025-10-15 17:43:39,063 - src.maritime_module.core.s57_data - INFO - Factory: Getting layer 'lndare'...
2025-10-15 17:43:39,112 - src.maritime_module.core.s57_data - INFO - Factory: Getting layer 'lndare'...
2025-10-15 17:43:39,647 - src.maritime_module.core.s57_data - INFO - Factory: Getting layer 'slcons'...
2025-10-15 17:43:39,886 - src.maritime_module.core.s57_data - INFO - Factory: Getting layer 'slcons'...
2025-10-15 17:43:39,938 - src.maritime_module.core.s57_data - INFO - Factory: Getting layer 'slcons'...
2025-10-15 17:43:39,991 - src.maritime_module.core.s57_d

## Save Graph

#### Save to GeoPackage (Most efficient)

In [14]:
# Time: 2m 48s - 1,428,704 nodes | 5,688,963 edges
# Time(h3): 5m 51s - 3,841,847 nodes | 11,533,570 edges
# Time(h3): 8m 18s - 3,838,809 nodes | 11,528,770 edges
# Time(h3): 10m 32s - 3,838,809 nodes | 11,528,770 edges
if save_gpkg:
    if graph_mode == "h3":
        name = f"{graph_mode}_graph_PG_6_11.gpkg"
        h3.save_graph_to_gpkg(G_h3, output_path=output_dir/ name)
    else:
        name = f"{graph_mode}_graph_{fg_space}.gpkg"
        fg.save_graph_to_gpkg(G_h3, output_path=output_dir / name)


2025-10-15 17:47:23,623 - pyogrio._io - INFO - Created 894,220 records
2025-10-15 17:47:26,583 - src.maritime_module.core.graph - INFO - Saved 894,220 nodes to /home/vikont_tux/python_projects_wsl2/1_MaritimeModule_V1/docs/notebooks/output/h3_graph_PG_6_11.gpkg in 7.257s
2025-10-15 17:48:43,028 - pyogrio._io - INFO - Created 2,673,606 records
2025-10-15 17:48:43,188 - src.maritime_module.core.graph - INFO - Saved 2,673,606 edges to /home/vikont_tux/python_projects_wsl2/1_MaritimeModule_V1/docs/notebooks/output/h3_graph_PG_6_11.gpkg in 37.980s
2025-10-15 17:48:43,189 - src.maritime_module.core.graph - INFO - === Graph Save Operation Performance Summary ===
2025-10-15 17:48:43,189 - src.maritime_module.core.graph - INFO - Timing Metrics:
2025-10-15 17:48:43,190 - src.maritime_module.core.graph - INFO -   nodes_processing_time: 16.314s
2025-10-15 17:48:43,190 - src.maritime_module.core.graph - INFO -   nodes_save_time: 7.257s
2025-10-15 17:48:43,190 - src.maritime_module.core.graph - INFO

#### Save to PostGIS (Original method)

In [15]:
# Time: 8m 54s - 1,428,704 nodes
# Time: 4m 25s - 1,428,704 nodes | 5,688,963 edges
# Time: 4m 17s - 1,428,704 nodes | 5,688,963 edges
# Time: 0m 56s - 356,996 nodes | 1,416,342 edges
if save_pg:
    if graph_mode == "h3":
        name = f"{graph_mode}_graph_PG_6_11"
        h3.save_graph_to_postgis(graph=G_h3, table_prefix= name, drop_existing=True)
    else:
        name = f"{graph_mode}_graph_{fg_space}"
        fg.save_graph_to_postgis(graph=G_fine, table_prefix= name, drop_existing=True)


2025-10-15 17:48:45,045 - src.maritime_module.core.graph - WARNING - PostgreSQL compatibility: table prefix 'h3_graph_PG_6_11' contains uppercase letters. Converting to lowercase: 'h3_graph_pg_6_11'. To avoid this warning, use lowercase identifiers.
2025-10-15 17:48:45,046 - src.maritime_module.core.graph - INFO - Saving graph to PostGIS schema 'graph' with tables: h3_graph_pg_6_11_nodes, h3_graph_pg_6_11_edges
2025-10-15 17:48:45,066 - src.maritime_module.core.graph - INFO - Ensured schema 'graph' exists.
2025-10-15 17:48:45,142 - src.maritime_module.core.graph - INFO - Dropped existing tables in 0.076s
2025-10-15 17:49:12,594 - src.maritime_module.core.graph - INFO - Saved 894,220 nodes to PostGIS in 14.549s
2025-10-15 17:50:56,051 - src.maritime_module.core.graph - INFO - Saved 2,673,606 edges to PostGIS in 58.317s
2025-10-15 17:51:03,873 - src.maritime_module.core.graph - INFO - Created spatial indexes in 7.821s
2025-10-15 17:51:04,287 - src.maritime_module.core.graph - INFO - Grap

#### Save to PostGIS (Optimized approach)

In [16]:
# Time: 2m 5s - 1,428,704 nodes | 5,688,963 edges (Chunk= 500 000)
# Time: 0m 38s - 356,996 nodes | 1,416,342 edges (Chunk= 500 000)

if save_pg_opt:
    if graph_mode == "h3":
        name = f"{graph_mode}_graph_opt_PG_6_11"
        h3.save_graph_to_postgis_optimized(graph=G_h3, table_prefix= name, drop_existing=True, chunk_size=500000)
    else:
        name = f"{graph_mode}_graph__opt_{fg_space}"
        fg.save_graph_to_postgis_optimized(graph=G_fine, table_prefix= name, drop_existing=True, chunk_size=500000)


2025-10-15 17:51:06,806 - src.maritime_module.core.graph - WARNING - PostgreSQL compatibility: table prefix 'h3_graph_opt_PG_6_11' contains uppercase letters. Converting to lowercase: 'h3_graph_opt_pg_6_11'. To avoid this warning, use lowercase identifiers.
2025-10-15 17:51:06,807 - src.maritime_module.core.graph - INFO - Saving graph to PostGIS (optimized) schema 'graph' with tables: h3_graph_opt_pg_6_11_nodes, h3_graph_opt_pg_6_11_edges
2025-10-15 17:51:06,808 - src.maritime_module.core.graph - INFO - Using chunk size: 500,000 records
2025-10-15 17:51:06,809 - src.maritime_module.core.graph - INFO - Schema and tables setup completed in 0.001s
2025-10-15 17:51:18,281 - src.maritime_module.core.graph - INFO - Processed 894,220 nodes in 11.472s
2025-10-15 17:51:30,058 - src.maritime_module.core.graph - INFO - Saved 894,220 nodes to PostGIS in 11.776s
2025-10-15 17:52:11,463 - src.maritime_module.core.graph - INFO - Processed 2,673,606 edges in 41.404s
2025-10-15 17:53:11,237 - src.marit

## Route

#### Create route (Base Route method)

In [22]:
calc_route = True
if calc_route:
    dep_point = port.get_port_by_name("Pilot")
    arr_point = port.get_port_by_name("San Francisco")

    route = Route(graph=G_h3, data_manager=pg_factory.manager) # Pass the data manager for saving/loading
    route_geometry, distance = route.base_route(
        departure_point=dep_point.geometry,
        arrival_point=arr_point.geometry,
    )

2025-10-15 18:09:42,178 - src.maritime_module.core.pathfinding_lite - INFO - Computing base route with Astar...
2025-10-15 18:09:42,183 - src.maritime_module.core.pathfinding_lite - INFO - Computing A* route...
2025-10-15 18:09:56,264 - src.maritime_module.core.pathfinding_lite - INFO - Mapped start point to graph node: (-122.26847761107062, 37.00108421407546)
2025-10-15 18:09:56,265 - src.maritime_module.core.pathfinding_lite - INFO - Mapped end point to graph node: (-122.41680297403626, 37.81661414529351)
2025-10-15 18:09:59,461 - src.maritime_module.core.pathfinding_lite - INFO - Successfully computed route with 801 nodes.
2025-10-15 18:09:59,463 - src.maritime_module.core.pathfinding_lite - INFO - Route computed successfully. Total distance: 62.43 nautical miles.


#### Plot Route

In [23]:
if calc_route:
    ply_fine_route = go.Figure(ply_fig)
    ply.add_route_trace(figure=ply_fine_route,
                        line=route_geometry,
                        name="Base Route")
    ply.add_single_port_trace(ply_fine_route, dep_point, name=dep_point['PORT_NAME'], color='blue')
    ply.add_single_port_trace(ply_fine_route, arr_point, name=arr_point['PORT_NAME'], color='red')
    ply_fine_route.show()